In [1]:
import OpenDartReader
import pandas as pd 

df = pd.read_csv('esg_finance_data.csv', dtype={'corp_code': 'str', 'stock_code': 'str'})
df['year'] = ['2020'] * 847 + ['2021'] * 847 + ['2022'] * 847
df['year'] = df['year'].astype('int')
print(df.shape, len(df.corp_code.unique())) #847
df.head(3)

(2541, 6) 847


,corp_name,esg,fin_state,corp_code,stock_code,year
0,기아,A+,"60,490,443,000,000",00106641,000270,2020
1,S-Oil,A+,"15,690,510,000,000",00138279,010950,2020
2,풀무원,A+,"1,693,573,176,270",00155355,017810,2020


In [18]:
# 매출액 데이터
api_key ='4fb4acc1d37b1dae74040e64a9674e018ccfd7fe'
# api_key = '1ee244d6168bde153cb2d45463d71139f567d1f4'
dart = OpenDartReader(api_key)

def sales_cash_docu(code, year):
    sales = dart.finstate(code, year)
    
    if sales.shape[1] >= 8:
        sale_df = sales.iloc[:, 8:]
    else:
        sale_df = sales
    # real_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')].iloc[0, :] # 매출액
    # c_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')]['thstrm_amount'].values # 매출액 # 문제1
    # p_sales = sales[(sales['fs_nm']=='연결재무제표')&(sales['account_nm']=='매출액')]['frmtrm_amount'].values # t-1기 매출액 # 문제 2

    div_cash = dart.report(code, '배당', year) #(연결)현금배당성향(%)
    if div_cash.shape[1] >= 3:
        cash_df = div_cash.iloc[:, 3:]
    else:
        cash_df = div_cash
    # c_div = div_cash[div_cash['se']=='(연결)현금배당성향(%)']['thstrm'].values # 순이익이 마이너스일 경우 0% # 배당금 / 당기순이익
    # p_div = div_cash[div_cash['se']=='(연결)현금배당성향(%)']['frmtrm'].values
    
    exec = dart.report(code, '임원', year) 
    # out_exec = exec[exec['ofcps'] == '사외이사'].shape[0]
    # num_exec = exec.shape[0]
    
    return sale_df, cash_df, exec


def others(code, year):
    holder = dart.report(code, '최대주주', year) # 보통주 계 + 우선수 계 (%) 
    # ratio_stakeholder = holder[(holder['stock_knd']=='보통주')&(holder['nm']=='계')].values # 문제3
    # out_exec = exec[exec['ofcps'] == '사외이사'].shape[0]
    # num_exec = exec.shape[0]
    employee = dart.report(code, '직원', year) # 총 직원 수

    return holder, employee


In [ ]:
# holder

dart.report('00106641', '최대주주', 2020)

# 2022 매출 데이터

In [ ]:
good_index, bad_index = [], []

for index, code in enumerate(df['corp_code'].unique()): 
    try:
        sales, cash, exec = sales_cash_docu(code, 2022)
        sales.to_csv(f'./2022/sales_2022/{code}_{str(2022)}.csv', encoding='utf8')
        cash.to_csv(f'./2022/cash_2022/{code}_{str(2022)}.csv', encoding='utf8')
        exec.to_csv(f'./2022/exec_2022/{code}_{str(2022)}.csv', encoding='utf8')
        print(f'{index}, Good!') # check if it works well
        good_index.append(index)
    except:
        print(f'{index}, bad...') # check if there is a problem
        bad_index.append(index)
        
print(len(good_index), len(bad_index))

In [ ]:
# good_index, bad_index = [], []

# for index, code in enumerate(df['stock_code'].unique()): 
#     try:
#         fin, inc = sales_docu(code, 2020)
#         fin.to_csv(f'./2020/fin_2020/{code}_{str(2020)}.csv', encoding='utf8')
#         inc.to_csv(f'./2020/inc_2020/{code}_{str(2020)}.csv', encoding='utf8')
          
#         print(f'{index}, Good!') # check if it works well
#         good_index.append(index)
#     except:
#         print(f'{index}, bad...') # check if there is a problem
#         bad_index.append(index)
        
# print(len(good_index), len(bad_index))

# 2020 매출 데이터

In [ ]:
good_index, bad_index = [], []

for index, code in enumerate(df['corp_code'].unique()): 
    try:
        sales, cash, exec, emp = sales_cash_docu(code, 2020)
        sales.to_csv(f'./2020/sales_2020/{code}_{str(2020)}.csv', encoding='utf8')
        cash.to_csv(f'./2020/cash_2020/{code}_{str(2020)}.csv', encoding='utf8')
        exec.to_csv(f'./2020/exec_2020/{code}_{str(2020)}.csv', encoding='utf8')
        emp.to_csv(f'./2020/emp_2020/{code}_{str(2020)}.csv', encoding='utf8')
        print(f'{index}, Good!') # check if it works well
        good_index.append(index)
    except:
        print(f'{index}, bad...') # check if there is a problem
        bad_index.append(index)
        
print(len(good_index), len(bad_index))

In [ ]:
# # importng data
# data, good_index, bad_index = [], [], []

# for index, (code , year) in enumerate(zip(df['stock_code'], df['year'])):
#     try:
#         data.append((code, year, sales_docu(code, year)))
#         good_index.append(index)
#     except:
#         bad_index.append(index)
# print(len(good_index), len(bad_index))

### 전체 데이터 

##### emp, holder 폴더는 각 회사 및 년도별로 구분하여 따져야함

In [19]:
good, bad = 0, 0

for code, year in zip(df['corp_code'], df['year']): 
    try:
        emp = dart.report(code, '직원', year)
        holder = dart.report(code, '최대주주', year)
        emp.to_csv(f'./emp/{code}_{str(year)}.csv', encoding='utf8')
        holder.to_csv(f'./holder/{code}_{str(year)}.csv', encoding='utf8')
        good += 1
        print(f'good: {good}')
    except:
        bad += 1
        print(f'bad: {bad}')

good: 1
good: 2
good: 3
good: 4
good: 5
good: 6
good: 7
good: 8
good: 9
good: 10
good: 11
good: 12
good: 13
good: 14
good: 15
good: 16
good: 17
good: 18
good: 19
good: 20
good: 21
good: 22
good: 23
good: 24
good: 25
good: 26
good: 27
good: 28
good: 29
good: 30
good: 31
good: 32
good: 33
good: 34
good: 35
good: 36
good: 37
good: 38
good: 39
good: 40
good: 41
good: 42
good: 43
good: 44
good: 45
good: 46
good: 47
good: 48
good: 49
good: 50
good: 51
good: 52
good: 53
good: 54
good: 55
good: 56
good: 57
good: 58
good: 59
good: 60
good: 61
good: 62
good: 63
good: 64
good: 65
good: 66
good: 67
good: 68
good: 69
good: 70
good: 71
good: 72
good: 73
good: 74
good: 75
good: 76
good: 77
good: 78
good: 79
good: 80
good: 81
good: 82
good: 83
good: 84
good: 85
good: 86
good: 87
good: 88
good: 89
good: 90
good: 91
good: 92
good: 93
good: 94
good: 95
good: 96
good: 97
good: 98
good: 99
good: 100
good: 101
good: 102
good: 103
good: 104
good: 105
good: 106
good: 107
good: 108
good: 109
good: 110
good: 11

In [17]:
print(good, bad)

2540 1
